In [1]:
%pip install python-dotenv jupysql duckdb duckdb-engine sqlalchemy openai twitter-api-client --quiet

Note: you may need to restart the kernel to use updated packages.


# Elementus API Usage Example

In [1]:
from elementus import ElementusClient, ElementusAPIError
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from helper import *
import os

# Load the environment variables
load_env()

# Retrieve Google BigQuery connection string from the environment
GBQ_URI = get_gbq_uri()
engine = create_engine(GBQ_URI)

# Retrieve the API key from the environment
ELEMENTUS_API_KEY = get_elementus_api_key()

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Retrieve last Bitcoin block transactions over 1 BTC

In [2]:
%load_ext sql
%sql engine

In [3]:
%%sql result <<
WITH max_block AS (
  SELECT MAX(number) as max_number 
  FROM `bigquery-public-data.crypto_bitcoin.blocks`
),
inputs AS (
  SELECT DISTINCT
    transaction_hash AS tx_hash,
    addresses
  FROM `bigquery-public-data.crypto_bitcoin.inputs`, max_block
  WHERE block_number = max_number
),
outputs AS (
  SELECT 
    transaction_hash AS tx_hash,
    addresses[OFFSET(0)] AS address,
    value / 100000000 AS value
  FROM `bigquery-public-data.crypto_bitcoin.outputs`, max_block
  WHERE block_number = max_number
)
SELECT 
    o.tx_hash, 
    o.address AS output_address, 
    o.value,
    ARRAY_AGG(DISTINCT input_address) AS input_addresses
FROM outputs o
JOIN inputs i ON o.tx_hash = i.tx_hash
CROSS JOIN i.addresses AS input_address
WHERE o.address NOT IN UNNEST(i.addresses) AND 
o.address NOT IN ('bc1qv4r4zwvf5ckdx3llaykl0vez4fdaj4jh5yfjsw') AND 
i.addresses[OFFSET(0)] NOT IN ('bc1qv4r4zwvf5ckdx3llaykl0vez4fdaj4jh5yfjsw') AND 
o.value >= 1
GROUP BY o.tx_hash, o.address, o.value
ORDER BY value DESC

Running query in 'bigquery://elementus-prod-242016'

92 rows affected.

In [4]:
df = result.DataFrame()
df

,tx_hash,output_address,value,input_addresses
0,3ba5fa8c6b7a9c70af2fd5e6bf9b159e3181c2a4a59ab9...,16scPoahbWZbz1hnLKJQ4KCz21nfMs2Rko,461.81202008,[1Af1SdEZiCNGX5TZwvoReGNpohkB1mrSJE]
1,3f55299ddf3cc3a0f43d5fd84bc3fb90083bd106654839...,17aoxXuaHDtRprwUMMEM9P4jUtQjPrEWX,84.84626999,[324H9uyTV9bPgAVgmdJNxPKKKZmowk5CYq]
2,72b6c5e42dd313cf30bc6c99693efa985c494eb3abc07a...,bc1qtprs2058jcrxk6jvg9a32rthwlh8l9u0kum384,78.99116759,[bc1qctkffmj8mg08002y483fkjhz7cjlm3g4m862rf]
3,3e5a81bfce3e4bf09d6733549686023ce9a2c6310d39cf...,bc1qz8z9wnx4cav4qam8wcdnvld8cm3w0rc7xxp60am24f...,75.76216739,"[3NH1eBb3AXQTQRCQkxZF8wYnm3s4vZPZy8, 3A35FrkSy..."
4,3e5a81bfce3e4bf09d6733549686023ce9a2c6310d39cf...,bc1pje9cuec0qhe85g9z40tzx9h6ylhtysne0t94e3wktu...,60.69195594,"[38E5R6QeEznyRyFcvBt6Bak1o793HYFpY2, 3NEpVCvMB..."
...,...,...,...,...
87,da610111bae26142eaf02b66998ff70a5cb308f8b104ed...,bc1qfjwdpq5zxsf980spjfqt4jf4l68af7vyynszy9,1.01569441,[15rPGbp32DBU4mEcAEtwcEcpx3ZxnUtp1Q]
88,3e5a81bfce3e4bf09d6733549686023ce9a2c6310d39cf...,bc1qqjl3d0ysvea7nwqanrwv7r22mlvgv88c7p0dm3,1,"[35CPFvrhRvyYNdVznBnhL8mkotxDnZ4RmS, bc1qrcdfa..."
89,72b6c5e42dd313cf30bc6c99693efa985c494eb3abc07a...,bc1pvxmfhg04v32vsdpzg98pu44vw73lljeyj009ff0f99...,1,[bc1qctkffmj8mg08002y483fkjhz7cjlm3g4m862rf]
90,40bfacbeaaa3113e7843fd71bbc3d3097e61547f814012...,bc1qlxvpzuxdk3h8m5yx6j0yhq45pw3exnceymg7f9,1,[3Msph4hwNgQX9SNujmoFBufkYvqYWCDztx]


### Use Elementus API to retrieve entity attributions for input and output addresses

In [5]:
client = ElementusClient(api_key=ELEMENTUS_API_KEY)

# Check API health
is_healthy = await client.check_health()

output_addresses = df['output_address'].unique().tolist()
input_addresses = list(set(addr for sublist in df['input_addresses'] for addr in sublist))

try:
    response = await client.get_address_attributions(output_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

# Create a dictionary from the response data
attribution_dict = {address: attr.entity for address, attr in response.data.items()}

try:
    response = await client.get_address_attributions(input_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

attribution_dict.update({address: attr.entity for address, attr in response.data.items()})

len(attribution_dict)

307

### Add entity attributions to transaction data

In [6]:
df['output_entity'] = df['output_address'].map(attribution_dict)
df['input_entity'] = [
    next((attribution_dict[key] for key in addresses if key in attribution_dict), None)
    for addresses in df['input_addresses']
]

df.dropna(subset=['input_entity', 'output_entity'], how='all', inplace=True)
df = df[df['input_entity'] != df['output_entity']]
# Ensure 'input_entity' is a string type before using str.startswith
df = df[~df['input_entity'].astype(str).str.startswith('fastmoney_') & ~df['output_entity'].astype(str).str.startswith('fastmoney_')]

df

,tx_hash,output_address,value,input_addresses,output_entity,input_entity
1,3f55299ddf3cc3a0f43d5fd84bc3fb90083bd106654839...,17aoxXuaHDtRprwUMMEM9P4jUtQjPrEWX,84.84626999,[324H9uyTV9bPgAVgmdJNxPKKKZmowk5CYq],cumberland.io,bullish.com
3,3e5a81bfce3e4bf09d6733549686023ce9a2c6310d39cf...,bc1qz8z9wnx4cav4qam8wcdnvld8cm3w0rc7xxp60am24f...,75.76216739,"[3NH1eBb3AXQTQRCQkxZF8wYnm3s4vZPZy8, 3A35FrkSy...",NaN,kraken.com
7,da610111bae26142eaf02b66998ff70a5cb308f8b104ed...,19tP9NDkXNZTUo8LNJmthc2tTEky8kwnmk,39.83989348,[15rPGbp32DBU4mEcAEtwcEcpx3ZxnUtp1Q],binance.com,None
10,da610111bae26142eaf02b66998ff70a5cb308f8b104ed...,18bYvustaVxkxA5wbh9D9adM4vGDCsB3Hd,33.29307079,[15rPGbp32DBU4mEcAEtwcEcpx3ZxnUtp1Q],binance.com,None
11,2a9480e8f6d02aa454ceb6c5428b7bd20ce1427317a6ce...,bc1qvzd00zz9jzkmwfnmswhmspryta0f6srmnnj3kr,24.91323426,"[3F1486ESEa6e7H3EJ4pRQtaEXNRhavjjy7, 34ePMddFX...",NaN,bitcoin.de
12,5fe4eb0a791e0ad00c6acf29c6bafa7bad6ac68fbb957d...,bc1q757dcrl8z9ljrzqscpz52l3eywwadald2mt3eg,24.16,"[32STz76CGkmMRMU3yUSQtYLT4DKuWhtSvG, bc1quhruq...",binance.com,okx.com
13,ed9c6d0cda11d4653df5c557f0c214599423ff2c5c4427...,bc1qw5y93pl3v6d4nt2r2ww68xhrhvyjmxqxg0fqqy,19.89999,"[1PEbgZszaAHu63Zc3BGVLHWQkEaHKHP1rR, bc1qdkmq7...",NaN,blockfills.com
17,8dba21254de4e0e91c17f80dcd8631d6680790ef60b109...,19tP9NDkXNZTUo8LNJmthc2tTEky8kwnmk,14.3396675,[bc1qrt7rkpswpgmcag7txzf6ps9mvepwgndshqdx6d],binance.com,kucoin.com
26,7966453ed3f53fd19d09caf356bd70461503586c8b1534...,bc1q88zfnu6wjqqpxyaksmc3mcgakwaqt7z68tkw0jx87g...,7.77,[bc1q5x9p784q6jkj4w3cw56h8vejz326yrf6u4tuksgg8...,NaN,okx.com
29,4165b01713c436b622a8312a931c266bebab02fbf799de...,bc1qqrkgdtcyupzzl8uyzf2wy8yp7qm59h007xr48h,6.72418184,[bc1qnkt02kg6e0xllp29u62z9lqxlgx2v6l9eq2nalgqd...,unknown_distributor_0013,None


In [7]:
# Convert DataFrame to JSON string
json_string = df[['tx_hash', 'input_entity', 'output_entity', 'value']].to_json(orient='records')

json_string

'[{"tx_hash":"3f55299ddf3cc3a0f43d5fd84bc3fb90083bd1066548390aa4b56c5cb3df0094","input_entity":"bullish.com","output_entity":"cumberland.io","value":84.84626999},{"tx_hash":"3e5a81bfce3e4bf09d6733549686023ce9a2c6310d39cfe0409c7d4db38d0f0e","input_entity":"kraken.com","output_entity":null,"value":75.76216739},{"tx_hash":"da610111bae26142eaf02b66998ff70a5cb308f8b104ed27b16491e9521968c2","input_entity":null,"output_entity":"binance.com","value":39.83989348},{"tx_hash":"da610111bae26142eaf02b66998ff70a5cb308f8b104ed27b16491e9521968c2","input_entity":null,"output_entity":"binance.com","value":33.29307079},{"tx_hash":"2a9480e8f6d02aa454ceb6c5428b7bd20ce1427317a6ce0e5dd18f1e1eb15d1c","input_entity":"bitcoin.de","output_entity":null,"value":24.91323426},{"tx_hash":"5fe4eb0a791e0ad00c6acf29c6bafa7bad6ac68fbb957d08b99541dedf109145","input_entity":"okx.com","output_entity":"binance.com","value":24.16},{"tx_hash":"ed9c6d0cda11d4653df5c557f0c214599423ff2c5c4427f47204f3da9c118e91","input_entity":"bl

### Create X Post using X AI to generate on-chain insights from transaction data

In [ ]:
from openai import OpenAI
from IPython.display import display, Markdown

OPENAI_API_KEY = get_openai_api_key()

client = OpenAI(
    api_key=OPENAI_API_KEY,
    #base_url="https://api.x.ai/v1",
    base_url="https://api.openai.com/v1",
)

completion = client.chat.completions.create(
    #model="grok-2-latest",
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "You are an expert in Bitcoin blockchain analytics and market structure"
        },
        {
            "role": "user",
            "content": "Given a list of Bitcoin transactions attributed to entities and news currently trending on Crypto Twitter create an X post explaining the most inportant on-chain insights, which may affect BTC/USD price. Provide only the text ready to be sent via X API without any additional comments. Limit the text to 160 characters. " + json_string
        },
    ],
    temperature=0.8,
)

display(Markdown(f"## X Post\n{completion.choices[0].message.content}"))


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x173dac770>


## X Post
Cumberland receives 84.85 BTC from Bullish, major inflows to Binance, Kraken outflows high—price impact potential. Stay alert! #Bitcoin #CryptoNews

### Tweet the generated post

In [10]:
from twitter.account import Account

account = Account(cookies={"ct0": "", "auth_token": ""})
account.tweet(text=completion.choices[0].message.content)

{'data': {'create_tweet': {'tweet_results': {'result': {'rest_id': '1890860377707073623',
     'has_birdwatch_notes': False,
     'core': {'user_results': {'result': {'__typename': 'User',
        'id': 'VXNlcjo5MDA3MjgwNTgyNTYxMzgyNDA=',
        'rest_id': '900728058256138240',
        'affiliates_highlighted_label': {},
        'has_graduated_access': True,
        'is_blue_verified': True,
        'profile_image_shape': 'Circle',
        'legacy': {'can_dm': True,
         'can_media_tag': True,
         'created_at': 'Thu Aug 24 14:34:38 +0000 2017',
         'default_profile': False,
         'default_profile_image': False,
         'description': 'We enable companies of all sizes to make better, faster decisions on the blockchain',
         'entities': {'description': {'urls': []},
          'url': {'urls': [{'display_url': 'elementus.io',
             'expanded_url': 'https://elementus.io/',
             'url': 'https://t.co/h7wrEp5WjF',
             'indices': [0, 23]}]}},
    